In [ ]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_circles
from mlxtend.plotting import plot_decision_regions

from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 12345

from qiskit_machine_learning.datasets import ad_hoc_data
train_data, train_labels, test_data, test_labels, sample_total = (
    ad_hoc_data(training_size=20, test_size=5, n=2, gap=0.3,
                include_sample_total=True, one_hot=False))

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(train_data, train_labels)

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
fig = plot_decision_regions(train_data, train_labels, clf=svm, ax=ax)
plt.legend(loc='upper left')
ax.set_aspect('equal')
plt.grid()
plt.show()

In [ ]:
X, Y = make_circles(n_samples=200, noise=0.05, factor=0.4)

A = X[np.where(Y==0)]
B = X[np.where(Y==1)]

A_label = np.zeros(A.shape[0], dtype=int)
B_label = np.ones(B.shape[0], dtype=int)

def make_train_test_sets(test_ratio=.3):
    def split(arr, test_ratio):
        sep = int(arr.shape[0]*(1-test_ratio))
        return arr[:sep], arr[sep:]

    A_label = np.zeros(A.shape[0], dtype=int)
    B_label = np.ones(B.shape[0], dtype=int)
    A_train, A_test = split(A, test_ratio)
    B_train, B_test = split(B, test_ratio)
    A_train_label, A_test_label = split(A_label, test_ratio)
    B_train_label, B_test_label = split(B_label, test_ratio)
    X_train = np.concatenate([A_train, B_train])
    y_train = np.concatenate([A_train_label, B_train_label])
    X_test = np.concatenate([A_test, B_test])
    y_test = np.concatenate([A_test_label, B_test_label])
    return X_train, y_train, X_test, y_test

train_data, train_labels, test_data, test_labels = make_train_test_sets()

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap

ZZFeatureMap(feature_dimension=2, reps=1).decompose().draw()

In [ ]:
qc = QuantumCircuit(2)
theta = Parameter('θ')
qc.cx(0, 1)
qc.rz(2*theta, 1)
qc.cx(0, 1)

qc.draw()

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

def calculate_kernel(zz_feature_map, x_data, y_data=None):
    if y_data is None:
        y_data = x_data
    sim = AerSimulator()
    x_matrix, y_matrix = [], []
    for x0, x1 in x_data:
        param0, param1 = zz_feature_map.parameters
        qc = zz_feature_map.bind_parameters({param0: x0, param1: x1})
        # .decompose() せずに .save_statevector() を使うとエラーになる。
        qc = qc.decompose()
        qc.save_statevector()
        sv = sim.run(qc).result().get_statevector()
        x_matrix.append(list(np.array(sv)))
    for y0, y1 in y_data:
        param0, param1 = zz_feature_map.parameters
        qc = zz_feature_map.bind_parameters({param0: y0, param1: y1})
        qc = qc.decompose()
        qc.save_statevector()
        sv = sim.run(qc).result().get_statevector()
        y_matrix.append(list(np.array(sv)))
    x_matrix, y_matrix = np.array(x_matrix), np.array(y_matrix)
    kernel = np.abs(
        y_matrix.conjugate() @ x_matrix.transpose()
    )**2
    return kernel

In [ ]:
zz_feature_map = ZZFeatureMap(feature_dimension=2, reps=2)
train_kernel = calculate_kernel(zz_feature_map, train_data)

In [ ]:
model = SVC(kernel='precomputed')
model.fit(train_kernel, train_labels);

In [ ]:
test_kernel = calculate_kernel(zz_feature_map, train_data, test_data)

pred = model.predict(test_kernel)

In [ ]:
import matplotlib

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
ax.set_aspect('equal')
ax.set_title("Predicted data classification")
ax.set_ylim(-2, 2)
ax.set_xlim(-2, 2)
for (x, y), pred_label in zip(test_data, pred):
    c = 'C0' if pred_label == 0 else 'C3'
    ax.add_patch(matplotlib.patches.Circle((x, y), radius=.01,
                 fill=True, linestyle='solid', linewidth=4.0,
                 color=c))
plt.grid()
plt.show()

In [ ]:
model.score(test_kernel, test_labels)